In [ ]:
from google.colab import drive

# Mount your Google Drive to access files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from tensorflow.keras.applications import DenseNet201, ResNet50
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from sklearn.metrics import accuracy_score

def load_images_and_labels_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(1 if "melanoma" in filename else 0)
    return np.array(images), np.array(labels)

# Load and preprocess images and labels
train_folder = '/content/drive/MyDrive/DL/sc/train'
test_folder = '/content/drive/MyDrive/DL/sc/test'

X_train, y_train = load_images_and_labels_from_folder(os.path.join(train_folder, 'benign'))
X_malignant, y_malignant = load_images_and_labels_from_folder(os.path.join(train_folder, 'malignant'))

# Combine benign and malignant data
X = np.concatenate([X_train, X_malignant])
y = np.concatenate([np.zeros(len(y_train)), np.ones(len(y_malignant))])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Augmentation for the training data
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Preprocess images for DenseNet201
X_train_preprocessed_densenet = densenet_preprocess_input(X_train.astype('float32'))
X_val_preprocessed_densenet = densenet_preprocess_input(X_val.astype('float32'))
X_test_preprocessed_densenet = densenet_preprocess_input(X_test.astype('float32'))

# Preprocess images for ResNet50
X_train_preprocessed_resnet = resnet_preprocess_input(X_train.astype('float32'))
X_val_preprocessed_resnet = resnet_preprocess_input(X_val.astype('float32'))
X_test_preprocessed_resnet = resnet_preprocess_input(X_test.astype('float32'))


In [ ]:

# Load pre-trained DenseNet201 model without the top (fully connected) layer
base_model_densenet = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model for feature extraction
for layer in base_model_densenet.layers:
    layer.trainable = False

# Add your own top layers for classification on top of DenseNet201
x_densenet = layers.GlobalAveragePooling2D()(base_model_densenet.output)
x_densenet = layers.Dense(512, activation='relu')(x_densenet)
x_densenet = layers.Dropout(0.5)(x_densenet)
predictions_densenet = layers.Dense(1, activation='sigmoid')(x_densenet)

# Create the model
model_densenet = Model(inputs=base_model_densenet.input, outputs=predictions_densenet)

# Compile the model
optimizer = Adam(learning_rate=0.0001)  # Adjust the learning rate as needed
optimizer = LossScaleOptimizer(optimizer)
model_densenet.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Use data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train_preprocessed_densenet)

# Train the model with data augmentation
model_densenet.fit(datagen.flow(X_train_preprocessed_densenet, y_train, batch_size=32), epochs=10, validation_data=(X_val_preprocessed_densenet, y_val))



74836368/74836368 [==============================] - 1s 0us/step
Epoch 1/10
56/56 [==============================] - 686s 12s/step - loss: 0.5779 - accuracy: 0.6830 - val_loss: 0.4405 - val_accuracy: 0.8085
Epoch 2/10
56/56 [==============================] - 645s 12s/step - loss: 0.4402 - accuracy: 0.7997 - val_loss: 0.3972 - val_accuracy: 0.8241
Epoch 3/10
56/56 [==============================] - 649s 12s/step - loss: 0.4186 - accuracy: 0.8103 - val_loss: 0.3707 - val_accuracy: 0.8486
Epoch 4/10
56/56 [==============================] - 644s 12s/step - loss: 0.3952 - accuracy: 0.8248 - val_loss: 0.3648 - val_accuracy: 0.8352
Epoch 5/10
56/56 [==============================] - 645s 12s/step - loss: 0.3834 - accuracy: 0.8309 - val_loss: 0.3612 - val_accuracy: 0.8552
Epoch 6/10
56/56 [==============================] - 638s 11s/step - loss: 0.3634 - accuracy: 0.8387 - val_loss: 0.3641 - val_accuracy: 0.8530
Epoch 7/10
56/56 [==============================] - 616s 11s/step - loss: 0.3572 - 

In [ ]:

# Load pre-trained ResNet50 model without the top (fully connected) layer
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model for feature extraction
for layer in base_model_resnet.layers:
    layer.trainable = False

# Add your own top layers for classification on top of ResNet50
x_resnet = layers.GlobalAveragePooling2D()(base_model_resnet.output)
x_resnet = layers.Dense(512, activation='relu')(x_resnet)
x_resnet = layers.Dropout(0.5)(x_resnet)
predictions_resnet = layers.Dense(1, activation='sigmoid')(x_resnet)

# Create the model
model_resnet = Model(inputs=base_model_resnet.input, outputs=predictions_resnet)

# Compile the model
model_resnet.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_resnet.fit(X_train_preprocessed_resnet, y_train, epochs=10, batch_size=32, validation_data=(X_val_preprocessed_resnet, y_val))


94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
56/56 [==============================] - 506s 9s/step - loss: 0.5563 - accuracy: 0.7450 - val_loss: 0.3820 - val_accuracy: 0.8330
Epoch 2/10
56/56 [==============================] - 500s 9s/step - loss: 0.3605 - accuracy: 0.8454 - val_loss: 0.3429 - val_accuracy: 0.8463
Epoch 3/10
56/56 [==============================] - 509s 9s/step - loss: 0.3331 - accuracy: 0.8566 - val_loss: 0.3221 - val_accuracy: 0.8508
Epoch 4/10
56/56 [==============================] - 516s 9s/step - loss: 0.2866 - accuracy: 0.8717 - val_loss: 0.3116 - val_accuracy: 0.8664
Epoch 5/10
56/56 [==============================] - 501s 9s/step - loss: 0.2780 - accuracy: 0.8756 - val_loss: 0.3010 - val_accuracy: 0.8731
Epoch 6/10
56/56 [==============================] - 504s 9s/step - loss: 0.2594 - accuracy: 0.8884 - val_loss: 0.3269 - val_accuracy: 0.8575
Epoch 7/10
56/56 [==============================] - 506s 9s/step - loss: 0.2344 - accurac

In [ ]:

# Extract features from DenseNet201
features_densenet_train = model_densenet.predict(X_train_preprocessed_densenet)
features_densenet_val = model_densenet.predict(X_val_preprocessed_densenet)
features_densenet_test = model_densenet.predict(X_test_preprocessed_densenet)

# Extract features from ResNet50
features_resnet_train = model_resnet.predict(X_train_preprocessed_resnet)
features_resnet_val = model_resnet.predict(X_val_preprocessed_resnet)
features_resnet_test = model_resnet.predict(X_test_preprocessed_resnet)

# Combine features from both models
X_train_combined = np.concatenate([features_densenet_train, features_resnet_train], axis=1)
X_val_combined = np.concatenate([features_densenet_val, features_resnet_val], axis=1)
X_test_combined = np.concatenate([features_densenet_test, features_resnet_test], axis=1)



13/13 [==============================] - 76s 6s/step


In [ ]:

# Create an SVM model
svm_model = make_pipeline(StandardScaler(), SVC(C=1, kernel='linear', random_state=42))

# Fit the SVM model on the training data
svm_model.fit(X_train_combined, y_train)

# Predict on the validation set
y_val_pred = svm_model.predict(X_val_combined)

# Evaluate accuracy on the validation set
accuracy_val = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy_val}")

# Predict on the test set
y_test_pred = svm_model.predict(X_test_combined)

# Evaluate accuracy on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {accuracy_test}")

Validation Accuracy: 0.8708240534521158
Test Accuracy: 0.8661616161616161
